# Live Air Quality Index (AQI)

Sanjeev - Data-Alchemist

## Load data

In [1]:
#%run ../aqi_dashboard/utils/get_aqi_data.py
import os
import pandas as pd
import utils.queries as q
from sqlalchemy import create_engine
from dotenv import load_dotenv
load_dotenv()

user = os.getenv("SNOWFLAKE_USER")
password = os.getenv("SNOWFLAKE_PASSWORD")
account = os.getenv("SNOWFLAKE_ACCOUNT")
warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
database = os.getenv("SNOWFLAKE_DATABASE")
schema = os.getenv("SNOWFLAKE_SCHEMA")

engine = create_engine(
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

In [9]:
df_aqi_data = pd.read_sql(q.get_latest_aqi_data(), engine)
df_aqi_data.head()

,date_fk,location_fk,aqi,aqi_category,prominent_pollutant,pm10_avg_value,pm25_avg_value,so2_avg_value,no2_avg_value,nh3_avg_value,...,o3_avg_value,country,state,city,station,latitude,longitude,record_ts,aqi_year,aqi_month
0,5277550057657283888,1643226973504317834,124.0,Moderate,CO,13,14,13,7,4,...,9,India,Odisha,Bileipada,"Tata Township, Bileipada - OSPCB",22.061567,85.474096,2025-07-07 02:00:00,2025,7
1,5277550057657283888,5157278151908002754,114.0,Moderate,PM2.5,65,31,26,16,2,...,17,India,Bihar,Bhagalpur,"DM Office_Kachari Chowk, Bhagalpur - BSPCB",25.251013,86.989001,2025-07-07 02:00:00,2025,7
2,5277550057657283888,3464970948614270270,124.0,Moderate,CO,40,24,4,12,4,...,37,India,Maharashtra,Latur,"Sawe Wadi, Latur - MPCB",18.399630,76.574520,2025-07-07 02:00:00,2025,7
3,5277550057657283888,8639546247816176272,174.0,Moderate,CO,23,11,2,12,8,...,24,India,Maharashtra,Kolhapur,"Shivaji University, Kolhapur - MPCB",16.687045,74.250587,2025-07-07 02:00:00,2025,7
4,5277550057657283888,-5283417601327970386,112.0,Moderate,CO,46,39,6,43,3,...,36,India,Chhattisgarh,Korba,"Urja Nagar, Korba - CECB",22.348441,82.549611,2025-07-07 02:00:00,2025,7


In [ ]:
unique_years = df_aqi_data["record_ts"].dt.year.unique().tolist()
unique_years
max_aqi = df_aqi_data[df_aqi_data.aqi_month == 7]["aqi"].max()
print("Max AQI:", max_aqi)
month_list = df_aqi_data["aqi_month"].unique().tolist()
month_list
states = df_aqi_data["state"].unique().tolist()
states

Max AQI: 500.0


['Odisha',
 'Bihar',
 'Maharashtra',
 'Chhattisgarh',
 'Chandigarh',
 'Delhi',
 'Rajasthan',
 'Uttar_Pradesh',
 'Gujarat',
 'Punjab',
 'Kerala',
 'TamilNadu',
 'Telangana',
 'Himachal Pradesh',
 'Karnataka',
 'Jammu_and_Kashmir',
 'Puducherry',
 'Madhya Pradesh',
 'Nagaland',
 'Andhra_Pradesh',
 'Jharkhand',
 'Assam',
 'Tripura',
 'Manipur',
 'West_Bengal',
 'Uttarakhand',
 'Sikkim',
 'Haryana',
 'Mizoram',
 'Meghalaya',
 'Arunachal_Pradesh']

## Plots

### Heatmap

In [ ]:
import altair as alt

month_list = df_aqi_data["aqi_month"].unique().tolist()
# selected_month = st.selectbox('Select a year', month_list)
selected_month=7
df_selected_month = df_aqi_data[df_aqi_data.aqi_month == selected_month]
df_selected_month_sorted = df_selected_month.sort_values(by="aqi", ascending=False)

# color_theme_list = ['blues', 'cividis', 'greens', 'inferno', 'magma', 'plasma', 'reds', 'rainbow', 'turbo', 'viridis']
# selected_color_theme = st.selectbox('Select a color theme', color_theme_list)
selected_color_theme="blues"

alt.themes.enable("dark")

def make_heatmap(input_df, input_y, input_x, input_color, input_color_theme):
    heatmap = alt.Chart(input_df).mark_rect().encode(
            y=alt.Y(f'{input_y}:O', axis=alt.Axis(title="Month", titleFontSize=18, titlePadding=15, titleFontWeight=900, labelAngle=0)),
            x=alt.X(f'{input_x}:O', axis=alt.Axis(title="", titleFontSize=18, titlePadding=15, titleFontWeight=900)),
            color=alt.Color(f'max({input_color}):Q',
                             legend=None,
                             scale=alt.Scale(scheme=input_color_theme)),
            stroke=alt.value('black'),
            strokeWidth=alt.value(0.25),
        ).properties(width=900
        ).configure_axis(
        labelFontSize=12,
        titleFontSize=12
        ) 
    # height=300
    return heatmap

heatmap = make_heatmap(df_aqi_data, 'aqi_month', 'state_geojson', 'aqi', selected_color_theme)

alt.Chart(...)

### Choropleth

In [18]:
# Choropleth via Altair
import altair as alt
from vega_datasets import data

alt.themes.enable("dark")

states = alt.topo_feature(data.us_10m.url, 'states')

alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('population:Q', scale=alt.Scale(scheme='blues')),   # scale=color_scale
    stroke=alt.value('#154360')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_selected_year, 'id', list(df_selected_year.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)



alt.Chart(...)

In [17]:
# Choropleth via Plotly
import plotly.express as px

choropleth = px.choropleth(df_selected_year, locations='states_code', color='population', locationmode="USA-states",
                               color_continuous_scale='blues',
                               range_color=(0, max(df_selected_year.population)),
                               scope="usa",
                               labels={'population':'Population'}
                              )
choropleth.update_layout(
        template='plotly_dark',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=0, b=0),
        height=350
    )

choropleth